# DeepSeek R1 训练思路在法律垂类上的实践

## 补充知识
- 策略模型 (Policy Model)
    - 也就是进行强化学习训练的模型，文中用的是 “Qwen2.5-7B-base”。
    - 每次采样输出时，会生成完整的回答。
- 判分模型 (Judge Model)
    - 文中用 “Qwen2.5-7B-instruct” 来对策略模型的输出做评价。
    - 作用：对策略模型的回答（输出）进行“对错/部分对”的判断，或者对刑期预测的误差进行评估。
    - 判分模型本质上是个“打分工具”，但和一般的“奖励模型”有一定区别：它不是直接给概率分值，而是输出一个文本（如 "2"、"1"、"0" 或者一个“误差：x个月”），再由预先设定的规则把这个结果转化成数值奖励。
- 奖励函数 (Reward Function / Rule-based Reward)
    - 负责把判分模型给出的文本化判分信息，映射成最终的数值奖励。
    - 先用正则或关键词去解析判分模型输出，然后结合“格式奖励”“刑期误差惩罚”等手动规则，把结果转换成一个具体的 reward 分数。
    - 因此，从架构上来讲，“奖励函数”并不是一个单独需要训练的模型，而是一系列“规则 + 对Judger输出的解析过程”。
    - 不同任务（罪名预测、相关条款预测、刑期预测等）会有不同的 reward 计算规则。

也就是说，“Qwen2.5-7B-instruct” 只负责把策略模型的输出与标准答案做比较，给出判断性的文本或数值信息，而真正用于计算和回传给策略模型的 reward 数值，则是由一套手动设计的规则（融合了“格式奖惩”和对judge model输出的解析逻辑”）来完成的。

简而言之：
- 判分模型（Qwen2.5-7B-instruct） ⇒ 输出“判定文本” (如 "2"、"1"、"0" 或者 “误差：6个月”)
- 奖励函数（rule-based) ⇒ 解析判定文本，转成实际的加减分 (如 +2、+1、0、-2、-误差/240 等)
- 策略模型（Qwen2.5-7B-base） ⇒ 接收最终的数值奖励进行强化学习更新